In [19]:
import random
from copy import deepcopy
import networkx as nx

In [20]:
def mst_edges(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    mst = nx.minimum_spanning_tree(G)
    return [tuple(sorted(e)) for e in mst.edges]

In [21]:
class Individual:
    def __init__(self, edges):
        self.edges = mst_edges(edges)
        self.fitness = 0
        self.calc_fitness()
    
    def calc_fitness(self):
        G = nx.Graph()
        G.add_edges_from(self.edges)
        self.fitness = sum(1 for node in G if G.degree[node] == 1)

In [22]:
def crossover(parent_1, parent_2):
    combined_edges = set(parent_1.edges).union(parent_2.edges)
    return Individual(list(combined_edges))

In [23]:
def mutation(all_edges, individual, mutation_prob):
    if random.random() < mutation_prob:
        u, v = random.choice(individual.edges)
        all_edges.remove((u,v))
        new_edges = mst_edges(all_edges)
        mst = nx.Graph()
        mst.add_edges_from(new_edges)
        if nx.is_connected(mst) and mst.number_of_nodes()==len(individual.edges)+1:
            individual.edges = mst.edges
            individual.calc_fitness()

In [24]:
def selection(population, tournament_size):
      participants = random.sample(population, tournament_size)
      return max(participants, key=lambda x: x.fitness)

In [25]:
def initialize_population(graph, population_size):
    for u, v in graph.edges():
        graph[u][v]['weight'] = random.randint(1, graph.number_of_nodes() + 1)

    population = []
    for _ in range(population_size):
        graph_perturbed = graph.copy()
        for u, v in graph_perturbed.edges():
            graph_perturbed[u][v]['weight'] += random.uniform(0.0, 1.0)
        population.append(Individual(nx.minimum_spanning_tree(graph_perturbed).edges))
        
    return population

In [26]:
def ga(graph, num_iters, elitism_size, mutation_prob, population_size, tournament_size):

    population = initialize_population(graph, population_size)
    new_population = deepcopy(population)
    
    if elitism_size % 2 != len(population) % 2:
        elitism_size += 1
    
    for _ in range(num_iters):
        population.sort(key=lambda x: x.fitness, reverse=True)
        new_population[:elitism_size] = deepcopy(population[:elitism_size])
    
        for i in range(elitism_size, population_size, 2):
            parent_1 = selection(population, tournament_size)
            parent_2 = selection(population, tournament_size)
            
            child_1 = crossover(parent_1, parent_2)
            child_2 = crossover(parent_1, parent_2)
            
            mutation(list(graph.edges), child_1, mutation_prob)
            mutation(list(graph.edges), child_2, mutation_prob)

            new_population[i] = child_1
            new_population[i + 1] = child_2
            
        population = deepcopy(new_population)

    return max(population, key=lambda x: x.fitness)